# 第14章 继承：瑕瑜互见

## 14.1 super()函数

- 坚持使用内置函数`super()`是确保面向对象的Python程序可维护性的基本要求。
- super提供两个参数：
    - type：从哪里开始搜索实现所需方法的超类。默认为`super()`调用所在的方法所属的类。
    - object_or_type：接收方法调用的对象（调用实例方法时）或类（调用类方法时）。在实例方法中调用`super()`时，默认为`self`。

## 14.2 子类化内置类型

内置类型（使用C语言编写）通常不调用用户定义的类覆盖的方法。

In [1]:
class DoppleDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [3]:
# dict的__init__方法忽略了覆盖的__setitem__方法
dd = DoppleDict(one=1)
dd

{'one': 1}

In [4]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [5]:
# update方法忽略了覆盖的__setitem__方法
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

## 14.3 多重继承和方法解析顺序

In [6]:
class Root:
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()

In [7]:
leaf1 = Leaf()
leaf1.ping()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [8]:
leaf1.pong()

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


唤醒过程由以下两个因素决定：
- Leaf类的方法解析顺序。
- 各方法中使用的`super()`。

## 14.4 多重继承的实际运用

1. ThreadingMixIn和ForkingMixIn

`ThreadingHTTPServer`多重继承了`socketserver.ThreadingMixIn`类和`HTTPServer`类，其中：
- `process_request_thread`方法没有调用`super()`。
- `process_request`方法：属于覆盖方法，启动一个线程，并把具体工作委托给运行在该线程中的`process_request_thread`，没有调用`super()`。
- `server_close`调用了`super().server_close()`以停止接收请求，然后等待`process_request`启动的线程完成工作。

2. Django泛化视图混入类

- `View`是所有视图的基类，提供了核心功能，例如`dispatch`方法。
- `TemplateView`类仅用于显示内容。
- `TemplateResponseMixin`类只适用于需要使用模板的视图，为`TemplateView`和其他渲染模板的视图提供行为。
- `ListView`是一个聚合类，实例化后，模板通过迭代`object_list`属性显示页面内容。

3. Tkinter中的多重继承

- Toplevel：表示`Tkinter`应用程序中顶层窗口的类。所有图形类中唯一没有继承`Widget`。
- Widget：窗口中所有可见对象的超类。直接继承自`BaseWidget`，并继承了`Pack`、`Place`和`Grid`（几何管理器，负责在窗口或窗体中排布小组件）。
- Button：普通的按钮小组件。只是`Widget`的子代，也间接继承`Misc`。
- Entry：单行可编辑文本字段。是`Widget`和`XView`（支持横向滚动）的子类。
- Text：多行可编辑文本字段。是`Widget`、`XView`和`YView`（支持纵向滚动）的子类。

## 14.5 应对多重继承

- 优先使用对象组合，而不是类继承。

- 理解不同情况下使用继承的原因：
    - 继承接口，创建子类型，实现“是什么”关系。
    - 继承实现，通过重用避免代码重复。
    

- 使用抽象基类显式表示接口。
    - 如果类的作用式定义接口，则显示地定义为抽象基类或者`typing.Protocol`的子类。
    - 抽象基类只能子类化`abc.ABC`或其他抽象基类。
    

- 通过混入明确重用代码。如果一个类的作用是提供方法，以供多个不相关的子类重用，但不体现“是什么”关系，那么就应该把那个类明确地定义为混入类。

- 为用户提供聚合类。

- 仅子类化为子类化设计的类。*子类化复杂的类并覆盖类方法容易出错，因为超类中的方法可能在不知不觉中忽略子类覆盖的行为*。

- 避免子类化具体类。具体类的实例通常有内部状态，覆盖依赖内部状态的方法时，很容易破坏状态。

## 14.6 杂谈

- 如果发现自己在构建多层类的层次结构，可能发生了以下事件：
    - 在重新发明轮子。去寻找框架或库，看看它们提供的组件是否可以在应用程序中复用。
    - 使用的框架设计不良，去寻找替代品。
    - 过度设计，需要遵守KISS原则。
    - 创造一个新框架。

- 内置类型dict、list和str是Python的底层基础，Cpython为了执行速度，不调用被子类覆盖的方法。

- 通过UserDict、UserList和UserString，可以加强扩展。

- 术语“面向对象”是Alan Kay发明的，而Smalltalk只支持单一继承。

- C++是第一门实现多重继承的流行语言。